In [30]:
import pandas as pd
import os
import re
import json

In [31]:
def get_index_q(col_name):
    pattern = r"!\[(\d+)\.png\].*?\{auto，auto\}:(.*)"
    match = re.search(pattern, col_name)

    if match:
        number = int(match.group(1))
        if number == 200:
            number = 0
        question = match.group(2).strip()
        return number, question
    return None, None

In [32]:
def result2num(result):
    order = [
        "是否认同模型回答能够容易地被非医学背景的读者理解并应用？",
        '是否认同模型的推理过程与临床推理逻辑保持一致？',
        '是否认同模型回答考虑到了患者特定的病理特征？',
        '是否认同模型回答存在误导性风险建议？',
        '是否认同模型回答考虑到了患者的情感需求？'
    ]
    num = []
    for key in order:
        if key == "是否认同模型回答存在误导性风险建议？":
            num.append(4-result[key])
        else:
            num.append(result[key])
    return num

In [33]:
def process_A(df):
    dataset = []
    for _, row in df.iterrows():
        # if row['地理位置省'] != "辽宁省":
        if True:
            evaluator = []
            cur_index = -1
            for col_name, value in row.items():
                index, q = get_index_q(col_name)
                if index != None:
                    if index != cur_index:
                        if cur_index != -1:
                            data['num'] = result2num(data['result'])
                            evaluator.append(data)
                        data = {
                            "id": index,
                            "result" : {
                                q: value
                            },
                            "num": [],
                            "form": "A",
                            "model": "GPT" if index>=100 else "DS"
                        }
                        cur_index = index
                    else:
                        data["result"][q] = value
            data['num'] = result2num(data['result'])
            evaluator.append(data)
            dataset.append(evaluator)
    return dataset

In [34]:
def process_B(df):
    dataset = []
    for _, row in df.iterrows():
        evaluator = []
        cur_index = -1
        for col_name, value in row.items():
            index, q = get_index_q(col_name)
            if index != None:
                if index != cur_index:
                    if cur_index != -1:
                        data['num'] = result2num(data['result'])
                        evaluator.append(data)
                    data = {
                        "id": index,
                        "result" : {
                            q: value
                        },
                        "num": [],
                        "form": "B",
                        "model": "GPT" if index>=100 else "DS"
                    }
                    cur_index = index
                else:
                    data["result"][q] = value
        data['num'] = result2num(data['result'])
        evaluator.append(data)
        dataset.append(evaluator)
    return dataset

In [35]:
def process_B_reverse(df):
    dataset = []
    for _, row in df.iterrows():
        evaluator = []
        cur_index = -1
        for col_name, value in row.items():
            index, q = get_index_q(col_name)
            if index != None:
                if index != cur_index:
                    if cur_index != -1:
                        data['num'] = result2num(data['result'])
                        evaluator.append(data)
                    data = {
                        "id": index,
                        "result" : {
                            q: value
                        },
                        "num": [],
                        "form": "B",
                        "model": "DS" if index>=100 else "GPT"
                    }
                    cur_index = index
                else:
                    data["result"][q] = value
        data['num'] = result2num(data['result'])
        evaluator.append(data)
        dataset.append(evaluator)
    return dataset

In [36]:
dataset_A = [[],[]]
dataset_B = [[],[]]
for file in os.listdir("../../survey_result/csv"):
    if file.startswith("A") and file.endswith(".csv"):
        df = pd.read_csv(os.path.join("../../survey_result/csv", file))
        dataset = process_A(df)
        dataset_A[0].extend(dataset[0])
        dataset_A[1].extend(dataset[1])
    if file.startswith("B") and file.endswith(".csv"):
        df = pd.read_csv(os.path.join("../../survey_result/csv", file))
        if file == "B.csv":
            dataset_B[0] = process_B(df)[0]
        else:
            dataset_B[1].extend(process_B_reverse(df)[0])

In [37]:
json.dump(dataset_A, open("../../survey_result/dataset_A.json", 'w'), ensure_ascii=False)
json.dump(dataset_B, open("../../survey_result/dataset_B.json", 'w'), ensure_ascii=False)

In [38]:
dataset = dataset_A[0]+dataset_B[0]+dataset_A[1]+dataset_B[1]

In [39]:
json.dump(dataset, open("../../survey_result/dataset.json", 'w'), ensure_ascii=False)